!pip install fredapi

!pip install yfinance

# 1. Data Cleaning and Processing

In [1]:
from fredapi import Fred
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
fred = Fred(api_key = "ed1b2123fc23b6e2a2d8db120bb68b7b")

Variable Name:

Initial claim (ICSA) (Weekly) : A claim filed by an unemployed individual after a separation from an employer. The claim requests a determination of basic eligibility for the Unemployment Insurance program.

cpi (CPIAUCSL) (Monthly):

term_srpead (T10Y3M) (Daily): 10-Year Treasury Yield minus 3-Month Treasury Yield

dgs10 (DGS10) (Daily): 10-Year Treasury Constant Maturity Rate

credit_spred (BAA10YM) (Monthly): Corporate Bond Yield minus 10-Year Treasury Yield 

sp500 (^GDPC) (Daily):

SGIXSENT (Daily): Sentiment Index

VIX :

VIX_retrun :


In [3]:
series_ids = {
    "initial_claims": "ICSA",
    "cpi": "CPIAUCSL",
    "term_spread": "T10Y3M",
    "dgs10": "DGS10",
    "credit_spread": "BAA10YM",
} 
data = {}
for name, sid in series_ids.items():
    s = fred.get_series(sid)
    s.name = name 
    data[name] = fred.get_series(sid)
    
df = pd.DataFrame(data)
df = df[(df.index >= "2018-01-01") & (df.index <= "2025-10-30")]
full_index = pd.date_range(start="2018-01-01", end="2025-10-30", freq="D")
df = df.reindex(full_index)
print(df.head(60)) 

            initial_claims      cpi  term_spread  dgs10  credit_spread
2018-01-01             NaN  248.859          NaN    NaN           1.68
2018-01-02             NaN      NaN         1.02   2.46            NaN
2018-01-03             NaN      NaN         1.03   2.44            NaN
2018-01-04             NaN      NaN         1.05   2.46            NaN
2018-01-05             NaN      NaN         1.08   2.47            NaN
2018-01-06        252000.0      NaN          NaN    NaN            NaN
2018-01-07             NaN      NaN          NaN    NaN            NaN
2018-01-08             NaN      NaN         1.04   2.49            NaN
2018-01-09             NaN      NaN         1.11   2.55            NaN
2018-01-10             NaN      NaN         1.13   2.55            NaN
2018-01-11             NaN      NaN         1.11   2.54            NaN
2018-01-12             NaN      NaN         1.12   2.55            NaN
2018-01-13        230000.0      NaN          NaN    NaN            NaN
2018-0

In [4]:
sp500 = yf.download("^GSPC", start="2018-01-01", end="2025-10-30")
sp500 = sp500["Close"]
sp500.name = "sp500"
print(sp500.head(), "\n")
print("Time Periods：", sp500.index.min(), "~", sp500.index.max())

C:\Users\admin\AppData\Local\Temp\ipykernel_29044\2462918612.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sp500 = yf.download("^GSPC", start="2018-01-01", end="2025-10-30")
[*********************100%***********************]  1 of 1 completed

Ticker            ^GSPC
Date                   
2018-01-02  2695.810059
2018-01-03  2713.060059
2018-01-04  2723.989990
2018-01-05  2743.149902
2018-01-08  2747.709961 

Time Periods： 2018-01-02 00:00:00 ~ 2025-10-29 00:00:00


In [5]:
df_sent = pd.read_excel("SGIXSENT_INDEX_20040115-20252030.xlsx",
                   sheet_name=["20140115_20240115", "20240115_20251030"])
df1 = df_sent["20140115_20240115"]
df2 = df_sent["20240115_20251030"]
def clean_sentiment(df):
    df = df.iloc[1:, 1:3].copy()
    df.columns = ["SGIXSENT", "Date"]
    df["Date"] = pd.to_datetime(df["Date"])
    df["SGIXSENT"] = pd.to_numeric(df["SGIXSENT"], errors="coerce")
    df.set_index("Date", inplace=True)
    return df

df_sent1 = clean_sentiment(df_sent["20140115_20240115"])
df_sent2 = clean_sentiment(df_sent["20240115_20251030"])
df_sent = pd.concat([df_sent1, df_sent2]).sort_index()
print(df_sent.head(10))
print(df_sent.tail(10))

            SGIXSENT
Date                
2014-01-15    243.05
2014-01-16    243.28
2014-01-17    244.07
2014-01-18       NaN
2014-01-19       NaN
2014-01-20       NaN
2014-01-21    244.22
2014-01-22    243.29
2014-01-23    243.48
2014-01-24    241.80
            SGIXSENT
Date                
2025-10-21    353.69
2025-10-22    352.76
2025-10-23    351.30
2025-10-24    350.91
2025-10-25       NaN
2025-10-26       NaN
2025-10-27    351.19
2025-10-28    352.30
2025-10-29    351.29
2025-10-30    349.96


In [6]:
df_sent = df_sent.loc["2018-01-01" : "2025-10-30"]
print(df_sent.head())
print(df_sent.tail())

            SGIXSENT
Date                
2018-01-01       NaN
2018-01-02    315.61
2018-01-03    315.61
2018-01-04    317.07
2018-01-05    318.38
            SGIXSENT
Date                
2025-10-26       NaN
2025-10-27    351.19
2025-10-28    352.30
2025-10-29    351.29
2025-10-30    349.96


In [7]:
vix = fred.get_series(
    "VIXCLS",
    observation_start="2018-01-01",
    observation_end="2025-10-30"
)
df_vix = pd.DataFrame(vix, columns=["VIX"])
df_vix.index = pd.to_datetime(df_vix.index)
print(df_vix.head(30))
print(df_vix.tail(30))

              VIX
2018-01-01    NaN
2018-01-02   9.77
2018-01-03   9.15
2018-01-04   9.22
2018-01-05   9.22
2018-01-08   9.52
2018-01-09  10.08
2018-01-10   9.82
2018-01-11   9.88
2018-01-12  10.16
2018-01-15    NaN
2018-01-16  11.66
2018-01-17  11.91
2018-01-18  12.22
2018-01-19  11.27
2018-01-22  11.03
2018-01-23  11.10
2018-01-24  11.47
2018-01-25  11.58
2018-01-26  11.08
2018-01-29  13.84
2018-01-30  14.79
2018-01-31  13.54
2018-02-01  13.47
2018-02-02  17.31
2018-02-05  37.32
2018-02-06  29.98
2018-02-07  27.73
2018-02-08  33.46
2018-02-09  29.06
              VIX
2025-09-19  15.45
2025-09-22  16.10
2025-09-23  16.64
2025-09-24  16.18
2025-09-25  16.74
2025-09-26  15.29
2025-09-29  16.12
2025-09-30  16.28
2025-10-01  16.29
2025-10-02  16.63
2025-10-03  16.65
2025-10-06  16.37
2025-10-07  17.24
2025-10-08  16.30
2025-10-09  16.43
2025-10-10  21.66
2025-10-13  19.03
2025-10-14  20.81
2025-10-15  20.64
2025-10-16  25.31
2025-10-17  20.78
2025-10-20  18.23
2025-10-21  17.87
2025-10-22

In [8]:
dif_vix = df_vix.sort_index()
print(df_vix.isna().sum())

VIX    49
dtype: int64


In [9]:
missing_dates =  df_vix[df_vix["VIX"].isna()].index
print(missing_dates)

DatetimeIndex(['2018-01-01', '2018-01-15', '2018-02-19', '2018-03-30',
               '2018-05-28', '2018-07-04', '2018-09-03', '2018-11-22',
               '2018-12-05', '2018-12-25', '2019-01-01', '2019-01-21',
               '2019-02-18', '2019-04-19', '2019-05-27', '2019-07-04',
               '2019-09-02', '2019-11-28', '2019-12-25', '2020-01-01',
               '2020-01-20', '2020-02-17', '2020-04-10', '2020-05-25',
               '2020-07-03', '2020-09-07', '2020-11-26', '2020-12-25',
               '2021-01-01', '2021-01-18', '2021-02-15', '2021-04-02',
               '2021-05-31', '2021-07-05', '2021-09-06', '2021-11-25',
               '2021-12-24', '2022-01-17', '2022-02-21', '2022-04-15',
               '2022-12-26', '2023-01-02', '2023-04-07', '2023-12-25',
               '2024-01-01', '2024-03-29', '2024-12-25', '2025-01-01',
               '2025-04-18'],
              dtype='datetime64[ns]', freq=None)


In [10]:
df_vix = df_vix.dropna(subset=["VIX"])

In [11]:
df_vix["VIX_return"] = np.log(df_vix["VIX"] / df_vix["VIX"].shift(1))

In [12]:
print(df_vix)

              VIX  VIX_return
2018-01-02   9.77         NaN
2018-01-03   9.15   -0.065563
2018-01-04   9.22    0.007621
2018-01-05   9.22    0.000000
2018-01-08   9.52    0.032020
...           ...         ...
2025-10-24  16.37   -0.055256
2025-10-27  15.79   -0.036074
2025-10-28  16.42    0.039123
2025-10-29  16.92    0.029996
2025-10-30  16.91   -0.000591

[1995 rows x 2 columns]


In [13]:
dif_vix = df_vix.sort_index()
print(df_vix.isna().sum())

VIX           0
VIX_return    1
dtype: int64


In [14]:
df_vix.index = pd.to_datetime(df_vix.index)
df_sent.index = pd.to_datetime(df_sent.index)
sp500.index = pd.to_datetime(sp500.index)
df.index = pd.to_datetime(df.index)

In [15]:
merged = pd.concat([df_vix, df_sent, sp500, df], axis=1, join="outer")
merged = merged.sort_index()
merged = merged.loc["2018-01-01" : "2025-10-30"]
print(merged.head(30))
print(merged.tail(60))
print(merged.info())

              VIX  VIX_return  SGIXSENT        ^GSPC  initial_claims      cpi  \
2018-01-01    NaN         NaN       NaN          NaN             NaN  248.859   
2018-01-02   9.77         NaN    315.61  2695.810059             NaN      NaN   
2018-01-03   9.15   -0.065563    315.61  2713.060059             NaN      NaN   
2018-01-04   9.22    0.007621    317.07  2723.989990             NaN      NaN   
2018-01-05   9.22    0.000000    318.38  2743.149902             NaN      NaN   
2018-01-06    NaN         NaN       NaN          NaN        252000.0      NaN   
2018-01-07    NaN         NaN       NaN          NaN             NaN      NaN   
2018-01-08   9.52    0.032020    318.38  2747.709961             NaN      NaN   
2018-01-09  10.08    0.057158    317.86  2751.290039             NaN      NaN   
2018-01-10   9.82   -0.026132    316.48  2748.229980             NaN      NaN   
2018-01-11   9.88    0.006091    315.86  2767.560059             NaN      NaN   
2018-01-12  10.16    0.02794

In [16]:
cols_ffill_others = ["SGIXSENT", "cpi", "credit_spread", "term_spread", "dgs10"]
merged_full = merged.copy()
merged_full[cols_ffill_others] = merged_full[cols_ffill_others].ffill()
cutoff = pd.Timestamp("2025-09-22")
s = merged["initial_claims"]
s_b = s.bfill()
s_f = s.ffill()
initial_new = s.copy()
mask_before = initial_new.index < cutoff
initial_new.loc[mask_before] = s_b.loc[mask_before]
mask_after = initial_new.index >= cutoff
initial_new.loc[mask_after] = s_f.loc[mask_after]
merged_full["initial_claims"] = initial_new
merged_trading = merged_full[merged_full["^GSPC"].notna()].copy()
check_cpi = merged_trading["cpi"].notna().astype(int).resample("ME").sum()
print(check_cpi[check_cpi == 0])

Series([], Freq: ME, Name: cpi, dtype: int64)


In [17]:
print(merged_trading.head(30))
print(merged_trading.tail(30))

              VIX  VIX_return  SGIXSENT        ^GSPC  initial_claims      cpi  \
2018-01-02   9.77         NaN    315.61  2695.810059        252000.0  248.859   
2018-01-03   9.15   -0.065563    315.61  2713.060059        252000.0  248.859   
2018-01-04   9.22    0.007621    317.07  2723.989990        252000.0  248.859   
2018-01-05   9.22    0.000000    318.38  2743.149902        252000.0  248.859   
2018-01-08   9.52    0.032020    318.38  2747.709961        230000.0  248.859   
2018-01-09  10.08    0.057158    317.86  2751.290039        230000.0  248.859   
2018-01-10   9.82   -0.026132    316.48  2748.229980        230000.0  248.859   
2018-01-11   9.88    0.006091    315.86  2767.560059        230000.0  248.859   
2018-01-12  10.16    0.027946    314.88  2786.239990        230000.0  248.859   
2018-01-16  11.66    0.137706    315.54  2776.419922        231000.0  248.859   
2018-01-17  11.91    0.021214    315.27  2802.560059        231000.0  248.859   
2018-01-18  12.22    0.02569

In [18]:
merged_trading.isna().any().any()
merged_trading.isna().sum()

VIX               0
VIX_return        1
SGIXSENT          0
^GSPC             0
initial_claims    0
cpi               0
term_spread       0
dgs10             0
credit_spread     0
dtype: int64

In [19]:
merged_trading["VIX_return"] = merged_trading["VIX_return"].fillna(0)

In [20]:
merged_trading.to_csv("merged_trading.csv", index=True)

I applied **front-fill** for `"SGIXSENT"`, `"cpi"`, and `"credit_spread"`.  

For `"Initial_Claims"`, I used **back-fill** before *2025-09-22*, since the data is published every Saturday (this is the most recent release available from FRED).  
After *2025-09-22*, I switched to **front-fill** for `"Initial_Claims"`.  

In addition, `"VIX_return"`  had no value on *2018-01-02* due to the calculation method, so I assigned it a value of **0** in the CSV file.

Also, I removed daily observations where the **U.S. stock market was closed**, keeping only the dates when the S&P 500 (`^GSPC`) had available data.

2025-10-13 (Columbus Day) :The stock market remains open, but the bond market is closed. So I frontfill **term_spread**, **dgs10** for this day.

In [21]:
merged_trading.index.to_series().diff().value_counts()

1 days    1538
3 days     355
4 days      53
2 days      21
Name: count, dtype: int64

In [22]:
# 1538 + 355 +53 + 21 # Confirm trading Days

In [23]:
merged_trading

,VIX,VIX_return,SGIXSENT,^GSPC,initial_claims,cpi,term_spread,dgs10,credit_spread
2018-01-02,9.77,0.000000,315.61,2695.810059,252000.0,248.859,1.02,2.46,1.68
2018-01-03,9.15,-0.065563,315.61,2713.060059,252000.0,248.859,1.03,2.44,1.68
2018-01-04,9.22,0.007621,317.07,2723.989990,252000.0,248.859,1.05,2.46,1.68
2018-01-05,9.22,0.000000,318.38,2743.149902,252000.0,248.859,1.08,2.47,1.68
2018-01-08,9.52,0.032020,318.38,2747.709961,230000.0,248.859,1.04,2.49,1.68
...,...,...,...,...,...,...,...,...,...
2025-10-23,17.30,-0.072455,351.30,6738.439941,218000.0,324.368,0.06,4.01,1.68
2025-10-24,16.37,-0.055256,350.91,6791.689941,218000.0,324.368,0.09,4.02,1.68
2025-10-27,15.79,-0.036074,351.19,6875.160156,218000.0,324.368,0.12,4.01,1.68
2025-10-28,16.42,0.039123,352.30,6890.890137,218000.0,324.368,0.10,3.99,1.68


In [24]:
# Z-score normalization for Sentiment (for XGB using)

merged_trading["SGIXSENT_z"] = (
    (merged_trading["SGIXSENT"] - merged_trading["SGIXSENT"].mean()) /
     merged_trading["SGIXSENT"].std()
)


# 2. GARCH Model

In [25]:
import pandas as pd
import numpy as np
from arch import arch_model
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")
import random
import os

# set seed
SEED = 42

np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

In [26]:
# VIX returns scale is too small, may hard to converge, but not necessary. 
# Our y scale = 0.0066, only 0.66% changes, but GARCH default scale is 1-1000. 
# It suggests x10 (returns x10), but I use raw data first.

#change first date of VIX returns to NA
merged_trading.loc["2018-01-02", "VIX_return"] = np.nan
returns = merged_trading["VIX_return"].dropna()

garch = arch_model(returns, vol="GARCH", p=1, q=1, dist="normal")
res = garch.fit(disp="off")

merged_trading["garch_resid"] = res.resid / res.conditional_volatility


In [27]:
# Create 5 Lag Features from GARCH residuals
for k in range(1, 6):
    merged_trading[f"resid_lag{k}"] = merged_trading["garch_resid"].shift(k)

# Drop first few rows with NaN lags
data = merged_trading.dropna().copy()

# Feature Set
feature_cols = ["resid_lag1","resid_lag2","resid_lag3","resid_lag4","resid_lag5",
                "SGIXSENT", "^GSPC", "initial_claims", "cpi", "term_spread", "dgs10", "credit_spread"]
target_col = "VIX_return"

X = data[feature_cols]
y = data[target_col]

In [28]:
merged_trading

,VIX,VIX_return,SGIXSENT,^GSPC,initial_claims,cpi,term_spread,dgs10,credit_spread,SGIXSENT_z,garch_resid,resid_lag1,resid_lag2,resid_lag3,resid_lag4,resid_lag5
2018-01-02,9.77,NaN,315.61,2695.810059,252000.0,248.859,1.02,2.46,1.68,-1.504842,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,9.15,-0.065563,315.61,2713.060059,252000.0,248.859,1.03,2.44,1.68,-1.504842,-0.530604,NaN,NaN,NaN,NaN,NaN
2018-01-04,9.22,0.007621,317.07,2723.989990,252000.0,248.859,1.05,2.46,1.68,-1.441465,0.086161,-0.530604,NaN,NaN,NaN,NaN
2018-01-05,9.22,0.000000,318.38,2743.149902,252000.0,248.859,1.08,2.47,1.68,-1.384598,0.014649,0.086161,-0.530604,NaN,NaN,NaN
2018-01-08,9.52,0.032020,318.38,2747.709961,230000.0,248.859,1.04,2.49,1.68,-1.384598,0.448157,0.014649,0.086161,-0.530604,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-23,17.30,-0.072455,351.30,6738.439941,218000.0,324.368,0.06,4.01,1.68,0.044437,-0.802429,0.396691,-0.144422,-0.914172,-1.516666,2.173576
2025-10-24,16.37,-0.055256,350.91,6791.689941,218000.0,324.368,0.09,4.02,1.68,0.027507,-0.643402,-0.802429,0.396691,-0.144422,-0.914172,-1.516666
2025-10-27,15.79,-0.036074,351.19,6875.160156,218000.0,324.368,0.12,4.01,1.68,0.039662,-0.447449,-0.643402,-0.802429,0.396691,-0.144422,-0.914172
2025-10-28,16.42,0.039123,352.30,6890.890137,218000.0,324.368,0.10,3.99,1.68,0.087846,0.566179,-0.447449,-0.643402,-0.802429,0.396691,-0.144422


# 3. XGBoost Traning with rolling window

In [29]:
train_size = int(len(data) * 0.8)  # 80% training period
predictions = []
true_values = []

for i in range(train_size, len(data)):
    X_train = X.iloc[:i]
    y_train = y.iloc[:i]
    X_test = X.iloc[i:i+1]

    model = XGBRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=3,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=SEED,
        n_jobs=1,              
        tree_method="exact"  
    )
    model.fit(X_train, y_train)
    pred = model.predict(X_test)[0]

    predictions.append(pred)
    true_values.append(y.iloc[i])

# XGBoost Hybrid Model Forecast Series
hybrid_pred = np.array(predictions)
true_values = np.array(true_values)

#To avoid look-ahead bias, we adopt a recursive rolling forecasting framework.
#At each time step t, the XGBoost model is re-trained only on information available prior to t, and generates a one-step-ahead forecast for VIX returns.
#This ensures that the hybrid model is evaluated under realistic real-time forecasting conditions.

# 4. Baseline GARCH Forecasting

In [30]:

def rolling_expanding_garch_forecast(returns, start_idx):
    """
    returns: pd.Series indexed by date
    start_idx: int, the index position where OOS forecasting begins (same as train_size)
    Returns: pd.Series aligned with returns.index (NaN before start_idx)
    """
    preds = [np.nan] * start_idx  # pad front
    # from start_idx to end-1, fit on returns[:i] and forecast i (1-step ahead)
    for i in range(start_idx, len(returns)):
        train_data = returns.iloc[:i]             # use all historical up to i-1
        model = arch_model(train_data, vol="GARCH", p=1, q=1, dist="normal")
        res = model.fit(disp="off")
        f = res.forecast(horizon=1).mean.iloc[-1, 0]  # 1-step-ahead mean
        preds.append(f)
    return pd.Series(preds, index=returns.index, name="garch_pred")

# train_size aligns with XGB 
train_size = int(len(data) * 0.8)
garch_series = rolling_expanding_garch_forecast(merged_trading["VIX_return"].dropna(), train_size)

#merged_trading.loc[garch_series.index, "garch_pred"] = garch_series



# 5. Models evaluation by RMSE

In [31]:
# Convert to arrays of equal length
garch_pred = garch_series.iloc[train_size:].values
hybrid_pred = hybrid_pred
true_vals = true_values

min_len = min(len(garch_pred), len(hybrid_pred), len(true_vals))
garch_pred = garch_pred[:min_len]
hybrid_pred = hybrid_pred[:min_len]
true_vals = true_vals[:min_len]


In [32]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

#Compute RMSE and MAE
rmse_garch = np.sqrt(mean_squared_error(true_vals, garch_pred))
rmse_hybrid = np.sqrt(mean_squared_error(true_vals, hybrid_pred))
mae_garch = mean_absolute_error(true_vals, garch_pred)
mae_hybrid = mean_absolute_error(true_vals, hybrid_pred)

print("RMSE - GARCH:", rmse_garch)
print("RMSE - Hybrid:", rmse_hybrid)
print("MAE  - GARCH:", mae_garch)
print("MAE  - Hybrid:", mae_hybrid)


RMSE - GARCH: 0.09041157320216787
RMSE - Hybrid: 0.09845520035524992
MAE  - GARCH: 0.05862037850019919
MAE  - Hybrid: 0.06272420304422699


Explanation:

- GARCH-only models inherently good at forecasting short-term VIX volatility (especially one-day ahead) because VIX exhibits strong volatility clustering and mean reversion—precisely the assumptions underlying GARCH models.

- The Hybrid model incorporates excessive noise characteristics (macro + sentiment daily alignment), which dilutes the signal.

- XGBoost is prone to overfitting minor real-time noise fluctuations → leading to poor long-term performance.

- our current approach is: forecasting a 1-day ahead short horizon → At this horizon, GARCH inherently holds an advantage.

- Therefore, results are not only reasonable but also an “expected outcome.”


In [33]:
# Which one performs better?
if rmse_hybrid < rmse_garch:
    print("\n Hybrid model improves forecast accuracy.")
else:
    print("\n Hybrid model does NOT outperform GARCH. Consider feature tuning.")


 Hybrid model does NOT outperform GARCH. Consider feature tuning.


In [34]:
#Direction accuracy
direction_garch = np.mean(np.sign(garch_pred) == np.sign(true_vals))
direction_hybrid = np.mean(np.sign(hybrid_pred) == np.sign(true_vals))

print("Direction Accuracy - GARCH:", direction_garch)
print("Direction Accuracy - Hybrid:", direction_hybrid)


Direction Accuracy - GARCH: 0.5597964376590331
Direction Accuracy - Hybrid: 0.48346055979643765


Direction Accuracy measures whether the model's predicted direction is correct.

Direction Accuracy=P(sign(pred)=sign(actual))

- if  > 0.5	Better than random (market predictability exists))

- if  = 0.5	Like flipping a coin, no predictive capability

- if  < 0.5	Reverse signals may exhibit exploitable patterns

Our hybrid model performs close to random guessing=0.5? 

##### Results Interpretation:
1. GARCH captures short-term volatility clustering → strong short-term predictive power.
2. Macro + sentiment factors operate at slower frequencies → hybrid model doesn’t help for 1-day horizon.
3. This suggests hybrid models may only outperform GARCH at longer horizons (e.g., 5-day / 22-day forecast).

# We should consider what should be our next step:
1. multi-horizon forecasting: using 5-day(1-week), 22-day(1-month) ahead RMSE + Direction Accuracy (we use 1-day ahead beforehand)
2. XGB Hyperparameter tuning 
3. including more features?

Those methods may improve the XGB forcasting, but due to the DS class criteria, we should definately keep the above (not ideal) results and explain it. Then, move on to the next step.


# 6. Multi-Horizon Forecasting (1-day, 1-week, 1-month)

In [35]:
# ==============================
# 6. Multi-Horizon Forecasting
# ==============================

horizons = [1, 5, 22]  # 1-day, 1-week, 1-month
results = {}

for h in horizons:
    print(f"\n---- Horizon = {h} days ahead ----")

    # Shift target for horizon forecast
    y_h = data[target_col].shift(-h).dropna()

    # Align X to y_h (because shifting reduces y length)
    X_h = X.loc[y_h.index]

    # Re-compute rolling train size under aligned dataset
    train_size_h = int(len(X_h) * 0.8)

    # ---- Hybrid XGB Rolling ----
    hybrid_preds_h = []
    true_vals_h = []

    for i in range(train_size_h, len(X_h)):
        X_train = X_h.iloc[:i]
        y_train = y_h.iloc[:i]
        X_test = X_h.iloc[i:i+1]

        model = XGBRegressor(
            n_estimators=200,
            learning_rate=0.05,
            max_depth=3,
            subsample=0.8,
            colsample_bytree=0.8,
            objective='reg:squarederror',
            random_state=SEED,
            n_jobs=1,
            tree_method="exact"
        )
        model.fit(X_train, y_train)
        pred = model.predict(X_test)[0]

        hybrid_preds_h.append(pred)
        true_vals_h.append(y_h.iloc[i])

    hybrid_preds_h = np.array(hybrid_preds_h)
    true_vals_h = np.array(true_vals_h)

    # ---- GARCH Rolling Forecast (multi-horizon mean forecast) ----
    garch_preds_h = []
    y_g = y_h.copy()

    for i in range(train_size_h, len(y_g)):
        train_data = y_g.iloc[:i]  
        model = arch_model(train_data, vol="GARCH", p=1, q=1, dist="normal")
        res = model.fit(disp="off")
        f = res.forecast(horizon=h).mean.iloc[-1, 0]  # mean forecast at horizon h
        garch_preds_h.append(f)

    garch_preds_h = np.array(garch_preds_h)

    # ---- Evaluation ----
    min_len = min(len(garch_preds_h), len(hybrid_preds_h), len(true_vals_h))
    garch_preds_h = garch_preds_h[:min_len]
    hybrid_preds_h = hybrid_preds_h[:min_len]
    true_vals_h = true_vals_h[:min_len]

    rmse_g = np.sqrt(mean_squared_error(true_vals_h, garch_preds_h))
    rmse_h = np.sqrt(mean_squared_error(true_vals_h, hybrid_preds_h))
    mae_g = mean_absolute_error(true_vals_h, garch_preds_h)
    mae_h = mean_absolute_error(true_vals_h, hybrid_preds_h)
    dir_g = np.mean(np.sign(garch_preds_h) == np.sign(true_vals_h))
    dir_h = np.mean(np.sign(hybrid_preds_h) == np.sign(true_vals_h))

    results[h] = [rmse_g, rmse_h, mae_g, mae_h, dir_g, dir_h]

# Convert results to comparison dataframe
results_df = pd.DataFrame(results, 
                          index=["RMSE_GARCH","RMSE_HYBRID","MAE_GARCH","MAE_HYBRID","DIR_GARCH","DIR_HYBRID"]).T

print("\n\n====== Multi-Horizon Comparison ======")
print(results_df)


---- Horizon = 1 days ahead ----

---- Horizon = 5 days ahead ----

---- Horizon = 22 days ahead ----


====== Multi-Horizon Comparison ======
    RMSE_GARCH  RMSE_HYBRID  MAE_GARCH  MAE_HYBRID  DIR_GARCH  DIR_HYBRID
1     0.090399     0.101633   0.058608    0.066116   0.559796    0.460560
5     0.090480     0.096608   0.058617    0.064103   0.558673    0.482143
22    0.090532     0.100377   0.058507    0.065766   0.559278    0.463918


### Multi-Horizon Forecasting Results (1-day, 5-day, 22-day)

- Extended the baseline 1-day-ahead forecasting to multiple horizons (1, 5, and 22 days ahead) to examine how predictive performance changes over different time scales.  

- Results show that the GARCH model consistently outperforms the Hybrid XGBoost model at short horizons, reflecting volatility clustering in daily returns.  

- However, the Hybrid model slightly narrows the performance gap at longer horizons (5-day and 22-day), suggesting that macro and sentiment variables may provide incremental information over medium-term horizons.  

- These findings are consistent with financial theory: short-term returns are dominated by noise and volatility effects, while longer-term dynamics are more influenced by fundamentals and sentiment.


# 7. Hyperparameter Tuning for XGBoost

In [36]:
# ==============================
# 7. Hyperparameter Tuning for XGBoost (Post-analysis)
# ==============================

from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

print("\n\n====== Starting Hyperparameter Tuning (based on 1-day horizon) ======")

# 1. Use 1-day horizon data for tuning
h = 1
y_h = data[target_col].shift(-h).dropna()
X_h = X.loc[y_h.index]

train_size_h = int(len(X_h) * 0.8)
X_train = X_h.iloc[:train_size_h]
y_train = y_h.iloc[:train_size_h]

# 2. Define parameter search grid
param_grid = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 4, 5],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.7, 0.8, 1.0]
}

# 3. Initialize model
xgb_base = XGBRegressor(objective='reg:squarederror', random_state=SEED)

# 4. Run grid search
grid_search = GridSearchCV(
    xgb_base, 
    param_grid, 
    scoring="neg_mean_squared_error", 
    cv=3,
    verbose=1,
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("\nBest parameters found:")
print(best_params)

# 5. Evaluate tuned model using the same rolling scheme (for consistency)
print("\n\n====== Evaluating Tuned Model (1-day horizon) ======")
hybrid_preds_tuned = []
true_vals_tuned = []

for i in range(train_size_h, len(X_h)):
    X_train = X_h.iloc[:i]
    y_train = y_h.iloc[:i]
    X_test = X_h.iloc[i:i+1]

    model = XGBRegressor(
        **best_params,
        objective='reg:squarederror',
        random_state=SEED,
        n_jobs=1,
        tree_method="exact"
    )
    model.fit(X_train, y_train)
    pred = model.predict(X_test)[0]
    hybrid_preds_tuned.append(pred)
    true_vals_tuned.append(y_h.iloc[i])

hybrid_preds_tuned = np.array(hybrid_preds_tuned)
true_vals_tuned = np.array(true_vals_tuned)

rmse_tuned = np.sqrt(mean_squared_error(true_vals_tuned, hybrid_preds_tuned))
mae_tuned = mean_absolute_error(true_vals_tuned, hybrid_preds_tuned)
dir_tuned = np.mean(np.sign(hybrid_preds_tuned) == np.sign(true_vals_tuned))

print(f"RMSE (Tuned Hybrid): {rmse_tuned:.6f}")
print(f"MAE  (Tuned Hybrid): {mae_tuned:.6f}")
print(f"DIR  (Tuned Hybrid): {dir_tuned:.6f}")

# Optional: Store results for report comparison
results_df.loc["Tuned_1d_Hybrid"] = [np.nan, rmse_tuned, np.nan, mae_tuned, np.nan, dir_tuned]

print("\n\n====== Updated Results Summary ======")
print(results_df)





====== Starting Hyperparameter Tuning (based on 1-day horizon) ======
Fitting 3 folds for each of 243 candidates, totalling 729 fits

Best parameters found:
{'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}


====== Evaluating Tuned Model (1-day horizon) ======
RMSE (Tuned Hybrid): 0.092344
MAE  (Tuned Hybrid): 0.059767
DIR  (Tuned Hybrid): 0.478372


====== Updated Results Summary ======
                 RMSE_GARCH  RMSE_HYBRID  MAE_GARCH  MAE_HYBRID  DIR_GARCH  \
1                  0.090399     0.101633   0.058608    0.066116   0.559796   
5                  0.090480     0.096608   0.058617    0.064103   0.558673   
22                 0.090532     0.100377   0.058507    0.065766   0.559278   
Tuned_1d_Hybrid         NaN     0.092344        NaN    0.059767        NaN   

                 DIR_HYBRID  
1                  0.460560  
5                  0.482143  
22                 0.463918  
Tuned_1d_Hybrid    0.478372  
